In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

from itertools import cycle
from matplotlib import pyplot as plt

from simulation import SimulationMixin


In [2]:
fontsize = 20

plots_dir = "plots"

In [3]:
colors = [("red", "red"), ("green", "green"), ("blue", "blue"), ("orange", "orange")]
color_cycle = cycle(colors)

In [ ]:
# program = "graph_coloring"
program = "dijkstra_token_ring"
# program = "maximal_matching"
# graph_name = "graph_powerlaw_cluster_graph_n7"
graph_name = "implicit_graph_n10"
# graph_name = "graph_10"
# graph_name = "graph_10"
sched = 0
no_simulations = 10000
me = False
fault_interval = 1

# duong_mode = True
simulation_type = "controlled_at_node_amit_v2"
limits = 1000

selected_nodes = [0]

In [5]:
def get_filename(graph_name, sched, simulation_type, args, no_simulations, me, fault_interval):
    return f"{graph_name}__{sched}__{simulation_type}_args_{args}__{no_simulations}__{me}__{fault_interval}__limits_{limits}"

In [6]:
# filenames = [
#     get_filename(
#         graph_name,
#         sched,
#         SimulationMixin.RANDOM_FAULT_SIMULATION_TYPE,
#         "",
#         no_simulations,
#         me,
#         fault_interval,
#     )
# ]
filenames = []

filenames.extend(
    [
        get_filename(
            graph_name,
            sched,
            simulation_type,
            arg,
            no_simulations,
            me,
            fault_interval,
        )
        for arg in selected_nodes
    ]
)


In [7]:
dfs = [pd.read_csv(os.path.join("results", program, f"{fn}.csv")) for fn in filenames]

FileNotFoundError: [Errno 2] No such file or directory: 'results/maximal_matching/graph_10__0__controlled_at_node_amit_v2_args_0__10000__False__1__limits_1000.csv'

In [ ]:
max_steps = max(df['Steps'].max() for df in dfs)
max_steps

np.int64(100)

In [ ]:
bins = np.linspace(30, max_steps, max_steps+1)
# bin_centers = (bins[:-1] + bins[1:]) / 2
bin_centers = bins

In [ ]:
bins

array([ 30. ,  30.7,  31.4,  32.1,  32.8,  33.5,  34.2,  34.9,  35.6,
        36.3,  37. ,  37.7,  38.4,  39.1,  39.8,  40.5,  41.2,  41.9,
        42.6,  43.3,  44. ,  44.7,  45.4,  46.1,  46.8,  47.5,  48.2,
        48.9,  49.6,  50.3,  51. ,  51.7,  52.4,  53.1,  53.8,  54.5,
        55.2,  55.9,  56.6,  57.3,  58. ,  58.7,  59.4,  60.1,  60.8,
        61.5,  62.2,  62.9,  63.6,  64.3,  65. ,  65.7,  66.4,  67.1,
        67.8,  68.5,  69.2,  69.9,  70.6,  71.3,  72. ,  72.7,  73.4,
        74.1,  74.8,  75.5,  76.2,  76.9,  77.6,  78.3,  79. ,  79.7,
        80.4,  81.1,  81.8,  82.5,  83.2,  83.9,  84.6,  85.3,  86. ,
        86.7,  87.4,  88.1,  88.8,  89.5,  90.2,  90.9,  91.6,  92.3,
        93. ,  93.7,  94.4,  95.1,  95.8,  96.5,  97.2,  97.9,  98.6,
        99.3, 100. ])

In [ ]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_data[0][0]

array([  87,  226,   89,    0,  222,  115,    0,  288,  137,    0,  278,
        120,  284,    0,  136,  285,    0,  164,  278,    0,  163,  234,
        151,    0,  234,  153,    0,  233,  125,    0,  221,  154,  186,
          0,  136,  161,    0,  157,  138,    0,  125,  100,  123,    0,
        104,  113,    0,   76,  106,    0,   83,  122,   71,    0,  113,
         58,    0,  107,   43,    0,  114,   37,   83,    0,   29,  100,
          0,   23,   74,    0,   25,   93,   10,    0,   73,   11,    0,
         67,    8,    0,   53,    6,   71,    0,    5,   65,    0,    3,
         53,    0,    3,   63,    1,    0,   47,    0,    0,   45,    2,
       1079])

In [ ]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_df = []
for hd in hist_data:
    hist_df.append(
        pd.DataFrame({
            'Steps': bin_centers[:-1],
            'Count': hd[0]
        })
    )

In [ ]:
hist_df[0]

,Steps,Count
0,30.0,87
1,30.7,226
2,31.4,89
3,32.1,0
4,32.8,222
...,...,...
95,96.5,0
96,97.2,0
97,97.9,45
98,98.6,2


In [ ]:
df_merged = hist_df[0]
for i in range(1, len(hist_df)):
    df_merged = pd.merge(df_merged, hist_df[i], on=["Steps"], suffixes=(i-1, i))
df_merged

,Steps,Count
0,30.0,87
1,30.7,226
2,31.4,89
3,32.1,0
4,32.8,222
...,...,...
95,96.5,0
96,97.2,0
97,97.9,45
98,98.6,2


In [ ]:
df_merged.set_index('Steps', drop=True, inplace=True)
df_merged

,Count
Steps,
30.0,87
30.7,226
31.4,89
32.1,0
32.8,222
...,...
96.5,0
97.2,0
97.9,45


In [ ]:
def get_title():
    return f"Simulation - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"

In [ ]:
def get_label(simulation_type, node):
    return {
        SimulationMixin.RANDOM_FAULT_SIMULATION_TYPE: "Random Fault",
        SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE: "Controlled (amit v1) at node %s",
        SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE_AMIT_V2: "Controlled (amit v2) at node %s",
        SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE_DUONG: "Controlled (duong) at node %s",
        SimulationMixin.RANDOM_FAULT_START_AT_NODE_SIMULATION_TYPE: "Random started at node %s",
    }.get(simulation_type, simulation_type) % (node)

In [ ]:
def get_filename():
    return f"{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{''.join([str(i) for i in selected_nodes])}__{simulation_type}__{limits}"

In [ ]:
from scipy.optimize import curve_fit


def exp_func(x, a, b, c):
    return a * np.exp(-b * x) + c


# Fit the curve
popt, _ = curve_fit(exp_func, df_merged.index, df_merged["Count"], p0= [10, 0.1, 0])
y_fit = exp_func(df_merged.index, popt[0], popt[1], popt[2])
df_merged[f"exp_fit"] = y_fit

In [ ]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, linewidth=1)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

# ax.set_yscale("log")
for i, line in enumerate(ax.lines):
    if i >= 0:
        line_style = "solid"
        line.set_color(next(color_cycle)[0])
    else:
        line_style = "dashed"
        line.set_color("goldenrod")
    line.set_linestyle(line_style)

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)

ax.set_xlabel("Steps")
ax.set_ylabel("Count")

# labels = ["Random Fault"]
labels = []
labels.extend([get_label(simulation_type, n) for n in selected_nodes])
labels.append(
    f"$y = {popt[0]:.2f}e^{{{-popt[1]:.2f}x}} {'+' if popt[2]>=0 else ''} {popt[2]:.2f}$"
)

custom_lines = [
    mlines.Line2D(
        [],
        [],
        color=line.get_color(),
        # marker=marker,
        label=cat,
        linestyle=line.get_linestyle(),
    )
    for line, cat in zip(ax.lines, labels)
]
plt.rc("font", size=fontsize)
plt.legend(handles=custom_lines, fontsize=fontsize * 0.9)
file_path = os.path.join(
    plots_dir,
    program,
    f"{get_filename()}.png",
)


plt.savefig(
    file_path,
    bbox_inches="tight",
)
plt.close()

print(f"Saved plot(s) for {file_path}")

Saved plot(s) for plots/maximal_matching/maximal_matching__graph_10__0__10000__2__0__controlled_at_node_amit_v2__100.png


In [ ]:
df_merged

,Count,exp_fit
Steps,,
30.0,87,163.918399
30.7,226,160.897192
31.4,89,157.951252
32.1,0,155.078704
32.8,222,152.277719
...,...,...
96.5,0,53.685126
97.2,0,53.410142
97.9,45,53.142009


In [ ]:
# Save to file
cols = [
    get_label(simulation_type, n) for n in selected_nodes
]
cols.append("exp_curve")
# df_merged.index = df_merged.index.astype(int)
df_merged.to_csv(os.path.join("results", program, f"w_exp_{get_filename()}.csv"))